### Visualization

#### Setup Environment

In [ ]:
Project_Root = '/gdrive/MyDrive/CV_Project/'

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd -q $Project_Root

In [ ]:
!ls

In [ ]:
!pip install -r requirements.txt --upgrade

In [ ]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from decompose import DecomposeVAE
import torchvision.datasets as datasets
from utils import computeResidual, save_img_tensors_as_grid
import os

#### Visualize the residuals

In [ ]:
weight_path = "checkpoints/save_1_best.pth"
device = "cuda:0"

model_container = DecomposeVAE(weight_path=weight_path, device = device)
fullvae = model_container.getFullVAE()

In [ ]:
data_dir = './data'

transform = torchvision.transforms.ToTensor()
mnist_testset = datasets.MNIST(root=data_dir, train=False, download=False, transform=transform)

# Get 1 image from every class
class_single = {}
for img, label in mnist_testset:
    if class_single.get(label.item(), -1) == -1:
        class_single[label.item()] = img
    if len(class_single.keys()) == 10:
        break
    
sorted_list = sorted(class_single.items())
single_batch = [item for _, item in sorted_list]

if(len(single_batch) < 10):
    print("Not all classes are present")

single_batch = torch.stack(single_batch)

In [ ]:
residual_save = "images/"

if(not os.path.isdir(residual_save)):
    os.makedirs(residual_save)

fullvae.eval()
with torch.no_grad():
    pred = fullvae(single_batch)
    residual = computeResidual(pred, single_batch)
    save_img_tensors_as_grid(residual, 1, f"{residual_save}residual.png")

#### Visualize the Latent Space Residual